In [2]:
import time
import re
import codecs
import urllib2 as ul
import xml.etree.ElementTree as ET

def ReadSent(pageno,src='./han/'):
    dat=codecs.open(src+str(pageno)+'.d',encoding='utf-8')
    dat = dat.read()
    dat = Clean(dat)
    dat = dat.replace('?','.')
    dat = dat.replace('!','.')
    return [' '.join(i.split()) for i in dat.split('.')]
    
def Clean(sentence,comma=0):
    if comma==0:
        for i in ['\n','_','-','(',')','"','\'',u'▲','...','[',']',u'■','<','>','\r',u'▶',u'◀']:
            sentence=sentence.replace(i,' ')
    else:
        for i in ['\n','_','-','(',')','"','\'',u'▲','...','[',']',u'■','<','>',',','\r']:
            sentence=sentence.replace(i,' ')
    return sentence.strip()

def LoadDB():
    sent1=[]
    f = open('./lists/han.list')
    for line in f:
        tmp=ReadSent(int(line.split('.')[0]))
        for j in tmp:
            sent1.append(j)

    sent2=[]
    f = open('./lists/dat.list')
    for line in f:
        tmp=ReadSent(int(line.split('.')[0]),src='./dat/')
        for j in tmp:
            sent2.append(j)
        
    sent3=[]
    f = open('./lists/oro.list')
    for line in f:
        tmp=ReadSent(int(line.split('.')[0]),src='./oro/')
        for j in tmp:
            sent3.append(j)

    return sent1+sent2+sent3

def ShortSent(sent):
    return sorted(sent, key = lambda x: len(x))

def SearchDB(sent,token):
    matches = [i for i in sent if i.find(token)>0]
    return sorted(matches, key=lambda x: len(x))

def SearchDBList(sent,tokens):
    matches=[]
    for token in tokens:
        matches += [i for i in sent if i.find(token)>0]
    return sorted(matches, key=lambda x: len(x))

def Simplify(sent):
    return list(set(sent))

#a la https://github.com/mouuff/Google-Translate-API/blob/master/python/GoogleTranslate.py
def translate(to_translate, to_langage="auto", langage="auto"):
    to_translate=to_translate.encode('utf-8')
    agents = {'User-Agent':"Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1;SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727; .NET CLR 3.0.04506.30)"}
    before_trans = 'class="t0">'
    link = "http://translate.google.com/m?hl=%s&sl=%s&q=%s"% (to_langage, langage, to_translate.replace(" ", "+"))
    request = ul.Request(link, headers=agents)
    page = ul.urlopen(request).read()
    result = page[page.find(before_trans)+len(before_trans):]
    result = result.split("<")[0]
    return result

def LoadWordBank(wbank,path='./xml/'):
    tree = ET.parse(path+wbank)
    root = tree.getroot()
    mywords={}
    for j,k in zip(root.iter('Word'),root.iter('English')):
        mywords[j.text]=k.text
    return mywords

def WCount(sent,write=0,n=2000,name='freq.wlist'):
    mywords={}
    for i in sent:
        i=i.replace(',','')
        for j in i.split():
            try:
                mywords[j]+=1
            except KeyError:
                mywords[j]=1
    freq = sorted(mywords.keys(), key=lambda x: mywords[x])
    freq.reverse()
    if write==1:
        dat=codecs.open('./output/'+name,'w',encoding='utf-8')
        for i in freq[:n]:
            try:
                dat.write(i)
                dat.write(', '+translate(i))
                dat.write('\n')
            except UnicodeDecodeError:
                continue
        
    return mywords, freq
        
def LoadFreqList(name,src='./output/'):
    f = open(src+name)
    keys=[]
    vals=[]
    for line in f:
        tmp = line.decode('utf-8').split(',')
        keys.append(tmp[0])
        vals.append(int(tmp[1]))
    return keys,vals

def MergeFreqList(name1,name2):
    dic1={}
    keys,vals=LoadFreqList(name1)
    keys1,vals1=LoadFreqList(name2)
    for i,j in zip(keys,vals):
        dic1[i]=j
    for i,j in zip(keys1,vals1):
        try:
            dic1[i]+=j
        except KeyError:
            dic1[i]=j
    keyt=sorted(dic1.keys(), key=lambda x: dic1[x])
    keyt.reverse()
    valt=[dic1[i] for i in keyt]
    return keyt,valt
    
    
    

In [3]:
sent=LoadDB()

In [14]:
words24 = [u' 표정',u' 꼭 ', u' 일부', u' 요즘', u' 계획', u' 느낌',u' 얼마',u' 고개',u' 성격',u' 계속',u' 세기',u' 세워',u' 아내',u' 가족',u' 현재']
matches=[SearchDB(sent,word) for word in words24]
print [len(i) for i in matches]
for i,j in enumerate(words24):
    print j
    for k in matches[i][:15]:print k,translate(k)

[3033, 3243, 1340, 2425, 2061, 3900, 4498, 889, 1214, 7137, 479, 419, 737, 1561, 9243]
 표정
같은 표정 Such expressions
라는 표정 Of expression
다양한 표정 Various expressions
담담한 표정 Cool look
구리의 표정 An expression of copper
검토실 표정 The layout room look
심각한 표정 Serious look
커제의 표정 Expression of keoje
비장한 표정들 One expression of spleen
밝은 표정이다 A bright smile
무덤덤한 표정 Mudeomdeomhan expression
패자들의 표정 Faces of the losers
즐거운 표정들 Enjoy the look
라는 표정이다 An expression of
내 표정 어때요 What about my face
 꼭 
그때 꼭 와 Then come and
TV 꼭 보세요 Must See TV
우승을 꼭 하겠다 I will win by all means
하지만 꼭 이겨라 But exactly Win
다음엔 꼭 두세요 Next time just leave
바둑이 꼭 그러하다 Badugi is necessarily true
담배는 꼭 끊으세요 Please make sure to quit tobacco
이번에는 꼭 이기자 This time, just Win
이거 꼭 올려주세요 Be sure to Vegas
내년엔 꼭 우승하자 Let&#39;s just win next year.
중국에 꼭 놀러와요 I just hang out in China
내일은 꼭 이기겠다 Tomorrow I&#39;ll be sure
그날 꼭 뵙겠습니다~ That means we&#39;ll see you ~
그런 사람이 꼭 있다 Such people are just
신안팀 꼭 우승하세요 Sinan sure your team win
 일부
빙상관 일부 Any par

In [22]:
keyt,valt=MergeFreqList('han.VV','oro.VV')
for i,j in zip(keyt,valt[:250]):
    print i+u'다,',translate(i+u'다')
len(keyt)

되다, become
가다, go
있다, have
두다, put
주다, give
거두다, Not afford to
열리다, open
이기다, win
꺾다, break
받다, Receive
보이다, show
잇다, connect
벌어지다, Widen
지나다, pass by
잡다, Catch
나오다, come out
늘다, gain
오르다, climb
알다, know
말하다, speak
보다, see
대하다, treat
벌이다, Punishment
타다, Ride
위하다, for
통하다, pass
패하다, L is
듣다, Hear
세다, count
따르다, follow
가지다, have
올르다, All Garda
가리다, blind
끝나다, finish
올리다, raise
갈다, Sharpen
만들다, to make
스다, Suda
남다, left
당하다, encounter
쥐다, hold
살다, live
샬다, Syalda
치다, hit
닿다, reach
만나다, meet
이어지다, be connected
이루다, accomplished
그러다, Then
쓰다, write
나가다, go out
내다, Pay
펼치다, unfold
모르다, I do not know
강하다, Strong
달리다, run
불리다, Bulrida
나서다, launch
그리다, draw
밝히다, lighten up
태다, TEDA
보여주다, to show
맞다, Right
커다, Keoda
추다, dance
넘다, exceed
해지다, Haejida
물리치다, To defeat
치르다, pay
남기다, leave
주어지다, Given Jeddah
전하다, impart
마치다, finish
들어가다, go in
짓다, build
시작하다, Start
까다, hatch
알리다, inform
갖다, have
맡다, undertake
이르다, say
배우다, learn
나누다, share
지키다, Protect
끊다, Quit
잡히다, be caught
펼쳐지다, Unfold
쉬다, rest
빠지

3086

In [25]:
keyt,valt=MergeFreqList('han.NNG','oro.NNG')
for i,j in zip(keyt,valt[:500]):
    print i,', ',translate(i)
len(keyt)

 바둑 ,  Baduk
단 ,  But
전 ,  I&#39;m
김 ,  Kim
한국 ,  Korea
배 ,  ship
국 ,  soup
강 ,  river
중국 ,  China
대국 ,  Superpower
돌 ,  stone
우승 ,  Championship
팀 ,  team
대회 ,  Competition
원 ,  circle
이세 ,  Ise
기사 ,  Article
시 ,  city
리그 ,  League
흑 ,  Black
박 ,  foil
오 ,  Five
선수 ,  player
패 ,  tile
일 ,  Work
훈 ,  Hoon
창호 ,  Windows and doors
승리 ,  Victory
시간 ,  time
결승 ,  finals
본선 ,  Main Line
위 ,  top
윤 ,  Yoon
대 ,  versus
세계 ,  World
기원 ,  origin
영 ,  spirit
한 ,  One
랭킹 ,  ranking
박정 ,  Bakjeong
일본 ,  Japan
연 ,  year
프로 ,  Pro
환 ,  ring
상금 ,  reward
제 ,  My
라운드 ,  round
오후 ,  afternoon
후 ,  after
판 ,  plate
집 ,  House
때 ,  time
진출 ,  debouchment
철한 ,  Cheolhan
현 ,  prefecture
장 ,  chapter
상대 ,  opponent
출전 ,  Exhibits
지석 ,  Whetstone
불계승 ,  Bulgyeseung
연승 ,  Winning streak
조 ,  article
백 ,  back
이번 ,  this time
대결 ,  Showdown
승부 ,  Fight
경기 ,  game
정 ,  tablet
초단 ,  Ultrashort
점 ,  point
게임 ,  game
라 ,  la
구리 ,  Copper
상 ,  phase
예선 ,  elimination
결과 ,  result
진행 ,  Progress
입단 ,  Joined
여자 ,  W

28181

In [10]:
keys,vals=LoadFreqList('han.VV')
keys1,vals1=LoadFreqList('oro.VV')
for i,j,k,l in zip(keys,vals[:50],keys1,vals1):
    print i+u'다',j,translate(i+u'다'),k,l,translate(k+u'다')

 되다 39776 become 가 82978 go
두다 19587 put 있 72687 have
보다 8776 see 되 53970 become
꺾다 8572 break 두 23957 put
거두다 8076 Not afford to 주 19992 give
잇다 6619 connect 열리 11912 open
열리다 6174 open 이기 11175 win
벌어지다 5944 Widen 거두 10091 Not afford to
이기다 5935 win 받 8317 Receive
보이다 5795 show 보이 8159 show
받다 5692 Receive 나오 7005 come out
오르다 5074 climb 지나 6685 pass by
잡다 4923 Catch 세 6440 count
벌이다 4720 Punishment 늘 6211 gain
지나다 4535 pass by 잡 6095 Catch
내다 4192 Pay 말하 6051 speak
타다 4111 Ride 알 6009 know
늘다 4062 gain 꺾 5838 break
패하다 3849 L is 대하 5479 treat
나오다 3512 come out 벌어지 5417 Widen
따르다 3184 follow 남 5338 left
위하다 3070 for 통하 4939 pass
대하다 3039 treat 잇 4854 connect
올리다 3022 raise 오르 4820 climb
알다 2996 know 위하 4522 for
쥐다 2856 hold 듣 4459 Hear
스다 2790 Suda 가지 3864 have
말하다 2767 speak 타 3622 Ride
올르다 2662 All Garda 만들 3569 to make
이루다 2630 accomplished 끝나 3525 finish
쉬다 2619 rest 맞 3472 Right
듣다 2603 Hear 가리 3419 blind
당하다 2505 encounter 그러 3320 Then
통하다 2472 pass 갈 3299 Sharpen
샬다 2463 Syald

In [ ]:
keys,vals=LoadFreqList('han.VX')
keys1,vals1=LoadFreqList('oro.VX')
for i,j,k,l in zip(keys,vals[:50],keys1,vals1):
    print i,j,translate(i),k,l,translate(k)

In [4]:
for i,j in zip(keys,vals[:250]):
    print i,j,translate(i)

바둑 49594 Baduk
전 42848 I&#39;m
김 38049 Kim
한국 32974 Korea
패 32049 tile
일 31323 Work
국 31059 soup
배 28962 ship
강 24970 river
원 22343 circle
우승 21916 Championship
중국 21869 China
한 21585 One
팀 21291 team
돌 20480 stone
이세 19945 Ise
시 19689 city
리그 19093 League
대회 18959 Competition
박 18857 foil
대국 18170 Superpower
흑 17327 Black
기사 16526 Article
조 16058 article
백 16053 back
훈 15316 Hoon
창호 14510 Windows and doors
승리 13711 Victory
윤 13148 Yoon
선수 12625 player
시간 12581 time
영 12047 spirit
제 11716 My
결승 11392 finals
본선 11186 Main Line
세계 10990 World
연 10747 year
대 10385 versus
불계승 10326 Bulgyeseung
박정 10012 Bakjeong
게임 9849 game
라운드 9626 round
환 9415 ring
일본 9326 Japan
상금 9244 reward
후 9036 after
랭킹 9002 ranking
집 8984 House
기원 8735 origin
입력 8631 input
작성자 8612 Writer
철한 8477 Cheolhan
결과 8474 result
초단 8319 Ultrashort
출전 8314 Exhibits
현 8169 prefecture
진출 8146 debouchment
프로 8098 Pro
대결 8072 Showdown
지석 8044 Whetstone
연승 7950 Winning streak
경기 7896 game
상 7876 phase
판 7866 plate
승부 7857 Fight


In [212]:
mywords,freq=WCount(sent)

In [218]:
print len(mywords),max(mywords.values())
for i in freq[:250]:print i,mywords[i]#,translate(i)   

942193 84469
있다 84469
9단 75555
수 62277
있는 50378
이 48723
한 43633
이세돌 43009
이창호 40552
오후 38801
중국 36725
: 36514
9단이 35704
것 32591
그 30733
바둑 28244
두 27667
한국 27131
9단은 27029
등 26278
입력 25433
| 24330
첫 24057
이번 23561
것이다 21620
최철한 21244
바둑을 20884
한국기원 20112
더 19871
우승 19418
중 19018
모두 19007
LG배 18449
9단의 18449
우승을 18103
박정환 18097
일본 17871
○ 17870
것이 17862
했다 17833
의 17746
박영훈 17713
대회 17485
대국 17131
지난 17119
것은 16766
하지만 16759
5단 16748
9단과 16729
잘 16351
본선 16279
가장 16109
한다 15961
흑 15914
후 15803
백 15707
구리 15611
강동윤 15576
때 15510
그러나 15371
조훈현 15183
큰 15150
2 15125
오전 15101
좋은 15072
중국의 14980
결승 14904
할 14634
함께 14482
7단 14394
꺾고 14328
삼성화재배 14313
또 14209
3단 14130
1 14082
9단을 14031
승리를 13476
없다 13464
많이 13425
하는 13394
같은 13355
열린 13265
주 13215
11 13167
결과 13005
이후 12901
김지석 12859
하고 12807
다시 12538
vs 12459
/ 12351
많은 12280
대국을 12166
4단 12107
같다 12101
벌어진 12066
현재 11924
2단 11810
고 11755
조한승 11683
● 11643
3 11560
된다 11510
이어 11485
있었다 11250
초읽기 11206
8단 11163
을 11143
끝 11126
만에 10900
프린트｜스크

In [213]:
sent = LoadDB()
wbank=LoadWordBank('wordbank_absbeg.xml')



In [223]:
etc=SearchDB(sent,u' 등 ')

In [225]:
for i in etc[:25]:print i,translate(i)

5, 1 등 0 5, such as 1 0
com 등 후원한다 Sponsors such as com
는 등 계속 질문한다 Continue questions such as
다만 등 돌리고 잤지요 Etc. I just slept turn
바둑 스포츠화 등 중략 Go Omitted sports shoes, etc.
입국 수속 등 때문이다 Because immigration procedures, etc.
Šimara 등 3명이다 Three people, including a Šimara
한국바둑전사 등 다수출간 Go Korea and many warriors published
그리고 인터넷 등 동감이다 And the sympathy Internet
이번 사활앱 등 전투적으로 This vital apps, such as militant
농심신라면배 이창호 등 출격 Nongshim Shin Ramen ship sorties, including Yichang
규칙 등 다른 이유가 있는가 Are there other reasons why such rules,
라는 등 좋은 반응을 얻었다 That got a good response, etc.
후야오위 등 4명이 차지했다 Hu yaoyu etc. took four people
피망 등 여러 곳이 있습니다 Bell peppers, etc. There are many places
’ 는 등 욕설들이 넘쳐난다 &#39;Etc. are filled with profanity
국가대표 신진서 등 4명이다 Standing four people, including a representative of emerging countries
나이 제한 등 장애물이 많았다 Were many obstacles, such as age limits
국가시드 3장 등 총 9장이다 The total seed, including national chapters is Chapter 39
농심배 등 중국에 여러번 왔다 Nongsimbae been several t

In [215]:
[j*i for ]

1107

In [152]:
import xml.etree.ElementTree as ET
tree = ET.parse('wordbank_absbeg.xml')
root = tree.getroot()
print root.tag
mywords=[]
ewords = []
for j,k in zip(root.iter('Word'),root.iter('English')):
    print j.text, k.text
    mywords.append(j.text)
    ewords.append(k.text)

IOError: [Errno 2] No such file or directory: 'wordbank_absbeg.xml'

In [145]:
import xml.etree.ElementTree as ET
tree = ET.parse('wordbank_beg.xml')
root = tree.getroot()
print root.tag
mywords=[]
ewords = []
for j,k in zip(root.iter('Word'),root.iter('English')):
    print j.text, k.text
    mywords.append(j.text)
    ewords.append(k.text)

mywordbank
다녀오다 to visit/ to go to and come back from
덕분에 thanks to
까지 until/ even/ up to
도와주다 to help
신정  New Year's Day (the solar calendar)
여행 trip
기다리다 to wait
몸 상태 health condition
저쪽 over there
부모님 parents
들다 to eat (honorific)
당연히 of course/ naturally 
어쩔 수 없다 can't help it/ to have no other choice
상사 company/ firm
날 day
끝나다(끝내다) to finish
많다 many
저야말로 as for me
고향 hometown
연락 contact
로비 lobby
잠시만 기다려주세요. Wait a moment please.
직접 directly/ personally
좋았겠어요 that must be good
참석하다 to attend
부탁 favor
이렇게 like this
기대하다 to look forward to
괜찮다 to be okay/ to be alright
좋아지다 to be better
전화 주다 to give a call
시 o'clock
올라가다 to go up/ to climb
연말 year-end
정상 top/ peak
뒤편 in the back of
꼭 surely/ certainly/ absolutely
언제든지 anytime
안내 guidance/ guide
아름답다 beautiful
부터 from
사무실 office
챙기다 to take/ to not forget/ to take without forgetting
스키장 ski resort
바다 ocean/ sea
부르다 to call
시간 내다 to make time
다녀오겠습니다 See you later (lit. I'll go and come back.)
잘 다녀와요 have a good day/ take care
18번 the

In [76]:
len(mywords)

1246

In [80]:
for i,j in zip(mywords[:100],ewords):
    matches=Simplify(SearchDB(sent,i))
    print i, translate(i),'(kclass)',j,len(matches)

다녀오다 Oda visited (kclass) to visit/ to go to and come back from 5
덕분에 Thanks to (kclass) thanks to 768
까지 Till (kclass) until/ even/ up to 51503
도와주다 help (kclass) to help 2
신정  theocracy (kclass) New Year's Day (the solar calendar) 2
여행 Travel (kclass) trip 1125
기다리다 wait (kclass) to wait 47
몸 상태 Body condition (kclass) health condition 54
저쪽 Beyond (kclass) over there 74
부모님 parents (kclass) parents 792
들다 Hold (kclass) to eat (honorific) 1279
당연히 naturally (kclass) of course/ naturally  1362
어쩔 수 없다 Can not help it (kclass) can't help it/ to have no other choice 146
상사 superior (kclass) company/ firm 208
날 day (kclass) day 34244
끝나다(끝내다) Terminate (finish) (kclass) to finish 0
많다 Many (kclass) many 3027
저야말로 That yamalro (kclass) as for me 0
고향 Hometown (kclass) hometown 1053
연락 Communication (kclass) contact 657
로비 lobby (kclass) lobby 624
잠시만 기다려주세요. Please wait. (kclass) Wait a moment please. 0
직접 directly (kclass) directly/ personally 2538
좋았겠어요 I liked (kclass) that must be goo

In [84]:
for i in matches:print i

더 좀 도와드리지 못하는 것이 아쉽습니다
예산에 잡혀 있지 않으면 도와드리고 싶어도 방법이 없습니다
”“무슨 말씀을, 제 아버님을 찾아오셨다는데 무엇인들 못 도와드리겠어요
박병규 8단도 선생님을 도와드리는 게 좋고 가르치는 일도 적성에 맞아 참 즐겁다고 한다
양재호 9단과 뜻을 같이 한 김승준 8단은 양재호 사범님을 도와드리는 것 이외에도 스스로의 바둑 연구에도 몰두 할 수 있어 적극 참여했다 고 소감을 밝혔다


In [82]:
matches=SearchDB(sent,u'도와드리')

In [83]:
len(matches)

5

In [85]:
tree

In [91]:
children=root.getchildren()

In [92]:
len(children)

737

In [117]:
BegBus=[(i.text,k.text) for i,j,k in zip(root.iter('Word'),root.iter('labels'),root.iter('English')) if j.text=='BegBusiness']
for i in BegBus:
    print i[0],i[1]
    matches=Simplify(SearchDB(sent,i[0]))
    cnt=0
    while cnt<5 and len(matches)>0:
        print '    ', matches.pop(0)
        cnt+=1


다녀오다 to visit/ to go to and come back from
     바둑리그 한게임, 전지훈련을 다녀오다 작성자:이경민, 2008 07 23 20:11 입력 한게임팀 선수들이 제주도로 전지훈련을 다녀왔다
     아내 김 씨는 결혼 전엔 무뚝뚝한 스타일이어서 반은 포기했는데 결혼한 뒤 되레 다정다감해졌다 며 얼마 전엔 중국리그 참석차 중국에 다녀오다가 공항 면세점에서 명품 지갑을 깜짝 선물로 사왔다 고 말했다
     한자로 표기된 중국 음식에 대해 김지석이 해박한 모습을 보여 평소 한자 공부를 하는지 묻자, 따로 한자 공부를 한 것은 아닌데 중국에 자주 다녀오다보니 자연스레… 김지석
     만약 초읽기 중에 화장실을 다녀오다 시간패를 당해도 어쩔 수가 없다
     이제 초읽기 정지 는 없다 한국기원, 소속기사 대국내규 일부 변경 단신 박주성 2014 05 21 오후 04:55 프린트｜스크랩 이제 초읽기 중에 화장실을 다녀오다 시간패를 당해도 어쩔 수가 없다
덕분에 thanks to
     사람들은 나름대로 자신만의 색깔이 있는데, 여러 색을 좋아한 덕분에 나는 장점을 잘 찾아내는 편이다
     그 덕분에 빠른 길로 숙소에 도착할 수 있었던 한국선수단은, 시합을 앞두고 편안히 잠자리에 들 수 있었다
     응원군 흑 들이 미리 놓여 있는 덕분에 펼칠 수 있는 작전이다
     다행히 마지막에 상대방이 실수를 해 준 덕분에 이길 수 있었답니다
     많은 분들이 도와주신 덕분에 좋아지고 있다
까지 until/ even/ up to
     지금까지 3연승 중이다
     한국은 부동의1위 박정환에게 랭킹시드로 출전권을 부여한 가운데, 선발전을 통과한 최철한 민상연 백찬희, 와일드카드로 합류한 이세돌까지 진용을 갖췄다
     식장 들어가기 전까지는… 작성자:김광호, 2010 01 09 02:32 입력 2009바둑대상 수상자들의 소감 한마디 바둑대상은 바둑계의 1년을 최종 마무리하는 시간
     하지만 스웨는 지난 27일과 28일 박

In [122]:
verbs=[(i.text,k.text) for i,j,k in zip(root.iter('Word'),root.iter('Class'),root.iter('English')) if j.text=='verb']
for i in verbs:
    print i[0],i[1]
    matches=Simplify(SearchDB(sent,i[0]))
    cnt=0
    while cnt<5 and len(matches)>0:
        print '    ', matches.pop(0)
        cnt+=1


다녀오다 to visit/ to go to and come back from
     바둑리그 한게임, 전지훈련을 다녀오다 작성자:이경민, 2008 07 23 20:11 입력 한게임팀 선수들이 제주도로 전지훈련을 다녀왔다
     아내 김 씨는 결혼 전엔 무뚝뚝한 스타일이어서 반은 포기했는데 결혼한 뒤 되레 다정다감해졌다 며 얼마 전엔 중국리그 참석차 중국에 다녀오다가 공항 면세점에서 명품 지갑을 깜짝 선물로 사왔다 고 말했다
     한자로 표기된 중국 음식에 대해 김지석이 해박한 모습을 보여 평소 한자 공부를 하는지 묻자, 따로 한자 공부를 한 것은 아닌데 중국에 자주 다녀오다보니 자연스레… 김지석
     만약 초읽기 중에 화장실을 다녀오다 시간패를 당해도 어쩔 수가 없다
     이제 초읽기 정지 는 없다 한국기원, 소속기사 대국내규 일부 변경 단신 박주성 2014 05 21 오후 04:55 프린트｜스크랩 이제 초읽기 중에 화장실을 다녀오다 시간패를 당해도 어쩔 수가 없다
도와주다 to help
     백지희: 처음에는 동료 여자프로기사들이 못 나가는 경우에 짬짬이 도와주다가 본격적으로 시작한 것은 지난해 10월부터다
     옆에서 인터넷 해설 진행을 도와주다 운 좋게 그의 입담들을 전해들을 수 있었는데 필자 혼자만 담아두기엔 아까운 어록들이 많아 지면을 빌어 독자들에게 간략히 소개한다
기다리다 to wait
     이동훈이 침착하게 기다리다가 기회를 승리로 연결시킨 바둑이다 고 총평했다
     그러나 입단 대회를 기다리다가 지쳐버렸다
     아무런 생각 없이 한 수 쉼을 계속 누르며 기다리다 종착점에 도달했는데 어이없게도 반집 패의 결과가 나왔다
     계시원은 아홉까지 말하고, 잠시 기다리다 어쩔 수 없이 작은 목소리로 열 을 불렀다
     ^^;; 상대가 안 나타나자 기다리다 살짝 잠든 녀석
끝나다(끝내다) to finish
참석하다 to attend
     알려진 대로 휴직 직접 사유는 기사총회, 바둑리그 불참은 피로 누적 때문 질

In [146]:
labels=list(set([i.text for i in root.iter('Class')]))
print labels


['proper noun', 'adjective', 'preposition', 'phrase', None, 'particle, preposition', 'adverb', 'phrase, verb', 'Verb', 'Noun', 'postposition', 'Verb (Honorific)', 'verb', 'conjunction, phrase', 'conjunction', 'slang', 'noun', 'determiner', 'particle', 'counter', 'Phrase', 'expression']


In [147]:
swords = [[(i.text,k.text) for i,j,k in zip(root.iter('Word'),root.iter('Class'),root.iter('English')) if j.text==z] for z in labels]

In [77]:
for i,j in zip(mywords[:10],ewords):
    matches=Simplify(SearchDB(sent,i))
    print i, translate(i),'(kclass)',j,len(matches)
    

어디 where (kclass) where 2341
기차 train (kclass) train 365
있습니까 do you have it (kclass) do you have? 121
시간 time (kclass) time 23796
단 But (kclass) sweet 269595
어떤 what (kclass) which 6157
더 more (kclass) more 45595
나중에는 lately (kclass) later/ after some time 68
는/은 It is / are (kclass) topic marking particle 0
은 silver (kclass) topic marking particle 413974


In [118]:
f=open('wordbank_beg.xml')

In [148]:
for i,j in enumerate(labels):
    print j, len(swords[i])
    if j==None:j='none'
    g=codecs.open(j+'.beg','w',encoding='utf-8')
    for k in swords[i]:
        g.write(k[0]+','+k[1]+'\n')

proper noun 6
adjective 24
preposition 2
phrase 20
None 346
particle, preposition 1
adverb 24
phrase, verb 1
Verb 1
Noun 1
postposition 1
Verb (Honorific) 1
verb 85
conjunction, phrase 1
conjunction 1
slang 2
noun 196
determiner 1
particle 3
counter 1
Phrase 1
expression 18


In [149]:
g=codecs.open('complete.beg','w',encoding='utf-8')
for i,j in enumerate(labels):
    print j, len(swords[i])
    if j==None:j='none'
    for k in swords[i]:
        g.write(k[0]+','+k[1]+'\n')

proper noun 6
adjective 24
preposition 2
phrase 20
None 346
particle, preposition 1
adverb 24
phrase, verb 1
Verb 1
Noun 1
postposition 1
Verb (Honorific) 1
verb 85
conjunction, phrase 1
conjunction 1
slang 2
noun 196
determiner 1
particle 3
counter 1
Phrase 1
expression 18


In [226]:
import nltk

In [227]:
from konlpy.tag import Kkma
from konlpy.utils import pprint
kkma = Kkma()

RuntimeError: Unable to load DLL [/Library/Java/JavaVirtualMachines/1.6.0.jdk/Contents/Libraries/libjvm.dylib], error = dlopen(/Library/Java/JavaVirtualMachines/1.6.0.jdk/Contents/Libraries/libjvm.dylib, 9): no suitable image found.  Did find:
	/Library/Java/JavaVirtualMachines/1.6.0.jdk/Contents/Libraries/libjvm.dylib: mach-o, but wrong architecture at native/common/include/jp_platform_linux.h:45

In [44]:
sent=LoadDB()
sent=list(set(sent))
sent=ShortSent(sent)

In [45]:
len(sent)

1213501

In [47]:
boda = [i for i in sent if i.find(u'보다 ')>0 and i.find(u' 더 ')>0]

In [48]:
len(boda)

2671

In [51]:
for i in boda[:100]:print i, translate(i)

예전보다 더 세졌다 Were three more than in the past
실전보다 더 진지하다 It is more serious than practical
3위보다 더 떨어졌다 3 was more down than up
8%보다 한참 더 크다 Ages greater than 8%
대국자보다 더 인기좋은 Players with more popular than good
바둑보다 더 소중한 것 Go to more valuable than
다른 때보다 더 기쁘다 More pleased than others
다른 대회보다 더 좋았다 Better than any other competition
강자 나보다 더 잘 둔다 Better placed than the leader or
그럼 외톨이보다 더 강자 So more than the solitary leader
4%의 조고보다 더 높다 4% higher than that of Zhao Gao
이보다 더 좋을 수 없다 It can not be better than this
이보다 더 궁금할 수 없다 There may be more than curious
고래심줄보다 더 질긴 인연 More than whale tendons tough ties
이보다 더 아플 수 있을까 What could be more painful
보통때보다 더 힘이 듭니다 This reduces more power than you normally
농구보다 바둑이 더 강하네 Badugi&#39;m stronger than basketball
작년보다 더 잘해야 할텐데 It would do better than last year
“지난 대국보다 더 떨렸다 &quot;I trembled more than last superpower
아들보다 더 좋아하는 아빠 More like a father than the son
이보다 더 화려할 수 없다 Not more than hwaryeohal
당연히 지금보다 더 기뻤다 Of course more than glad 

In [63]:
for k in range(6)[-1:]:
    g=codecs.open('./output/morethan_'+str(k)+'.list','w',encoding='utf-8')
    for i,j in enumerate(boda[k*500:min([len(boda)-1,(k+1)*500])]):
        try:
            j=Clean(j)
            g.write(j)
            g.write(',')
            g.write(translate(j))
            g.write('\n')
            if i%20==0:print k,i,j
        except UnicodeDecodeError:
            continue
    g.close()
    time.sleep(30)

5 0 그 동안 흥창배 준우승, 정관장배 우승 등 여러 세계대회에서 활약하며 매스컴에 얼굴을 비친 박지은이지만 여자 최초 9단으로 이름을 올린 것이 이전 대회 우승보다 훨씬 더 값진 것이라는 것을 깨달았을 것이다
5 20 또한 파크랜드와 동률이 나올 경우 승자승 원칙에 따라 1승 1무로 앞서고 있는 파크랜드에 포스트시즌 티켓을 내줘야 하며, 따라서 신성건설로써는 14라운드에서 파크랜드보다 개인 승수 단 1승이라도 더 얻어내야 한다
5 40 그러나 그 승부보다 더 깊게 남은 것은, 이길 수 있다는 자신감을 가지고 배짱있게 자신의 바둑을 당당히 펼쳐 보인 신민준 군과 그 어린 꼬마를 상대로 최선을 다해 대국하는 모습을 보여준 루이 나이웨이 9단의 모습이었다
5 60 바둑과 바둑 친구가 좋아 1년에도 몇 번씩 일본 중국 홍콩 대만 베트남 태국에다가 콩그레스가 열리는 유럽의 나라마다 찾아가는 김향희 씨는 “열 몇 시간 쪼그리고 앉아 한 번에 가는 것보다 중간에 내렸다가 타는 것이 더 재미있다
5 80 저자 윤4단은 처음으로 쓰는 책이라 부족한 부분이 많겠지만 독자들의 기력 향상에 도움이 된다면 그것보다 보람된 일은 없을 것이라며 영문으로도 번역된 만큼 해외 바둑인들이 한국바둑과 좀 더 가까워질 수 있는 계기가 되기를 희망한다고 밝혔다
5 100 1978년 4월부터 1978년 10월까지 일본 NHK에서 방영됐던 ‘미래소년 코난’은 핵무기 보다 더 무서운 위력을 지닌 초자력 무기로 전쟁을 치른 결과 지구는 대지각 변동을 일으켜 지축이 뒤틀어지고 여섯 대륙은 산산 조각난 채 바다 속에 가라앉아 버렸다
5 120 현재 추산으로는 ‘내곡지구안’이 확정될 경우 토지 수용가는 당초 매입가 평당 290만원 보다 다소 높은 평당 350만원선이 되겠지만, 몇 년 후 다시 불하 받을 때는 감정가가 평당 1,000만원선이 될 것으로 전망돼 앞으로 땅값으로만 100억원 정도가 더 필요할 것으로 예상된다
5 140 새로운 교재와 교수법 사용한 효과적인 교육방법 18세 이하 유럽 단체전 개최 

HTTPError: HTTP Error 400: Bad Request

In [69]:
geot = SearchDB(sent,u'것 같')
len(geot)

18917

In [71]:
for i in geot[:100]:print i,translate(i)


하는 것 같다 It seems that
했던 것 같다 It seems that
어떨 것 같아 How about I think
아닌 것 같다 Not seem to
그런 것 같다 It seems that
끝난 것 같다 It seems over
힘들 것 같아요 Hard to think
와전된 것 같다 It seems a misrepresentation
치열할 것 같다 It seems rages
“그럴 것 같다 &quot;I think so
대단한 것 같다 It seems great
죽일 것 같았다 It seemed to kill
난 질 것 같아 I think it will be
실수인 것 같다 It seems to be a mistake
서두른 것 같다 It seems awkward
자겁할 것 같어 Gateo will jageop
긴장한 것 같다 It seems tense
이길 것 같아요 I will win
좋아진 것 같다 It seems jotahjin
인 것 같습니다 It seems to be
괜찮은 것 같다 It seems good
끈질긴 것 같다 It seems insistent
쉬울 것 같은가 Equal it will be easier
어떤 것 같은가 Equal to anything
그랬을 것 같다 It seems geuraeteul
“그런 것 같다 &quot;Maybe
이었던 것 같다 It seems to have been
운명인 것 같다 It seems fate
좋은 것 같아요 I think good
욕들을 것 같음 Equal to the bath
불리한 것 같아요 I think unfavorable
안 그럴 것 같다 Do not seem to do that
그런것 같진 않다 Geureongeot is unlikely
음 맞는 것 같다 Well it seems to fit
패가 날 것 같다 Plaque seems to me
더 좋은 것 같나 Do you think better
재미있을 것 같아 It sounds fun
재미있을 것 같다 Sounds li

In [73]:
for k in range(6):
    g=codecs.open('./output/seems_'+str(k)+'.list','w',encoding='utf-8')
    for i,j in enumerate(geot[k*500:min([len(geot)-1,(k+1)*500])]):
        try:
            j=Clean(j)
            g.write(j)
            g.write(',')
            g.write(translate(j))
            g.write('\n')
            if i%20==0:print k,i,j
        except UnicodeDecodeError:
            continue
    g.close()
    time.sleep(30)

0 0 하는 것 같다
0 20 괜찮은 것 같다
0 40 경솔했던 것 같다
0 60 그게 통한 것 같다
0 80 1위를 할 것 같다
0 100 운이 따르는 것 같다
0 120 그래야 할 것 같았다
0 140 4국은 어떨 것 같나
0 160 올해 우승할 것 같다
0 180 뭔가 계획적인 것 같다
0 200 도 작용을 한 것 같다
0 220 음식도 파는 것 같아요
0 240 내 바둑 어떤 것 같아
0 260 이번도 그럴 것 같았다
0 280 좋은 승부인 것 같아요
0 300 좋은 컨디션인 것 같다
0 320 초반은 무난했던 것 같다
0 340 부담감은 아닌 것 같고…
0 360 그렇지는 않았던 것 같다
0 380 자매가 닮은 것 같습니까
0 400 그래서 전투형인 것 같다
0 420 너무 낙관했던 것 같아요
0 440 요는 실력인 것 같습니다
0 460 야경도 멋있었던 것 같다
0 480 어떤 대결이 될 것 같은가
1 0 내가 쉽게 물러설 것 같아
1 20 이번에도 그런 것 같은데…
1 40 소감이 특별할 것 같은데요
1 60 운이 좋아서 이긴 것 같다
1 80 “엄청나게 팍팍한 것 같다
1 100 왠지 변수가 될 것 같았다
1 120 기적을 바라야 할 것 같다
1 140 1시간반이면 괜찮을 것 같다
1 160 기막힌 그림이 나올 것 같다
1 180 오랜만에 효녀가 된 것 같다
1 200 사실 성공한 것 같지 않아요
1 220 네, 그렇게 된 것 같습니다
1 240 건강문제가 가장 큰 것 같다
1 260 마치 이렇게 말하는 것 같다
1 280 정말 감회가 남다를 것 같다
1 300 단지 개방해도 좋을 것 같다
1 320 부안과 인연이 있는 것 같다
1 340 색다른 경험이 될 것 같아요
1 360 사는 수가 있었던 것 같은데
1 380 O로 끝날 것 같은 분위기였다
1 400 아마도 그 말이 맞을 것 같다
1 420 금메달을 딸 것 같은 느낌이다
1 440 목9단의 밥 맛이 쓸 것 같다
1 460 운영비도 적지 않을 것 같은데
1 480 나만 잘 맞춰주면 될 것 같

In [86]:
chul=SearchDB(sent,u' 줄 알')
chul+=SearchDB(sent,u' 줄 모르')
chul+=SearchDB(sent,u' 줄 몰')
len(chul)

1574

In [83]:
chul=ShortSent(chul)
for i in chul[:100]:print i,translate(i)

죽은 줄 알았다 Found dead line
이긴 줄 알았어 Although knew
샌 줄 알았는데 I thought San
지는 줄 알았다 I knew that
이긴 줄 알았을까 Although line did she know
미치는 줄 알았다 I knew on
“넌 될 줄 알았어 &quot;I knew to be
하는 줄 알았다니깐 Line kicks knew that
“ 그러실 줄 알았다 &quot;I knew you&#39;d come
내가 졌는 줄 알았네 All right, I give jyeotneun
끝까지 둘 줄 알았다 Both knew the end
정말 지는 줄 알았다 I knew that
제가 진 줄 알았어요 I knew Jean
10초바둑인 줄 알겠다 I see the line go 10 seconds
그렇게 한가한 줄 알아 So give out free
9급 바둑인 줄 알겠네 Grade 9&#39;ll know the Monarch line
중앙 살아있는 줄 알았네 All right living central line
난투극이 될 줄 알았는데 I thought to be scuffle
거의 이기는 줄 알았는데 Although almost thought
그 때문에 질 줄 알았다 Consequently knew Quality
일찍부터 이럴 줄 알았어 I knew it early
조금 더 이긴 줄 알았다 Although I knew a little bit more
욕도 맞서 칠 줄 알고요 I also know how to play against York
정말 끝내기인 줄 알았다 I knew the finish
어디 이게 쉬운 줄 알았니 Where did I know this is an easy line
학생들이 또래인 줄 알겠다 I know that the students are peers
내가 많이 이긴 줄 알았어 Although I knew a lot
흑이 뚫고 나올 줄 알았다 I knew the game through the black

In [87]:
for k in range(3):
    g=codecs.open('./output/chul_'+str(k)+'.list','w',encoding='utf-8')
    for i,j in enumerate(chul[k*500:min([len(chul)-1,(k+1)*500])]):
        try:
            j=Clean(j)
            g.write(j)
            g.write(',')
            g.write(translate(j))
            g.write('\n')
            if i%20==0:print k,i,j
        except UnicodeDecodeError:
            print 'errored with',i,j
            continue
    g.close()
    time.sleep(30)

0 0 죽은 줄 알았다
0 20 일찍부터 이럴 줄 알았어
0 40 이번에 입단할 줄 알았거든요
0 60 컴퓨터에 능통한 줄 알고 있다
0 80 이번 대회에서 입단할 줄 알았어요
errored with 85 七縱七擒 18급 그럴 줄 알았어유
0 100 대마, 제가 가만히 둘 줄 알았습니까
0 120 운동도 할 겸 30분이 좋을 줄 알았다
0 140 어려운 상대인 줄 알지만 꼭 돌파해보겠다
0 160 사실 그때는 루이가 무조건 이길 줄 알았다
0 180 흑이 좋은 줄 알았는데 막상 만만치가 않았다
0 200 “실례인 줄 알지만 한 판만 더 기회를 주시죠
0 220 상을 받을 줄 알았으면 가족과 함께 왔을 텐데요
errored with 225 ~ ㅋㅋ 벳만 잘하는 줄 알았더니 센스도 굿입니다
errored with 236 중국인들은 한국인들이 漢字만을 사용하는 줄 알지요
0 240 연분홍편지 1급 손 빼길래 살아서 손 뺀 줄 알았네
0 260 그때는 제가 잘 두는 줄 알았고 약간 우쭐하기도 했다
0 280 국후 김 7단은 “상대가 던지려는 줄 알았다”고 토로
0 300 칼 마르크스의 유물론 영향으로 교회는 텅 빌 줄 알았는데
errored with 310 2단 옆차기를 할 줄 알아서 ㅋㅋㅋㅋㅋㅋㅋ 업글중 입니다
0 320 군대가서 녹이 슬었을 줄 알았더니, 여전히 날이 서 있네
0 340 중반에도 대세점을 하나쯤을 놓칠 줄 알았지만 정말 완벽했다
errored with 342 ㉴。ㄹ♪ 18급 성룡이 감독만 잘하는 줄 알앗는데 잘 두네
0 360 마지막에 역전당한 줄 알고 조마조마했는데 다행히 이겨 기쁩니다
0 380 하지만, 역전된 줄 알았던 바둑은 여전히 마효춘이 앞서고 있었다
0 400 흑97은 하변 백대마에 수가 나는 줄 알았다는 국후 안성준의 후회
0 420 “중반에 좋은 줄 알았는데 계가를 잘못하고 있었다가 나중에 역전했다
0 440 이번 선수 선발에서 저를 당연히 버리실 줄 알았는데, 또 택해 주셨다
errored with 458 나중에 정관계로 나가는 

In [2]:
sent=LoadDB()

In [5]:
acham=SearchDB(sent,u' 아참 ')

In [9]:
len(acham)
for i in acham:print i, translate(i)

 @:: 아참 고바야시는 스승의 딸과 결혼을 했었지요 Oh: @ Kobayashi and married the daughter of a teacher haeteotjiyo
ㅜ:: 아참 요즘에 영어몰입교육 이야기가 시중의 화제죠 TT: Oh&#39;s English education, the topic of the talk these days the market
ㅜ:: 아참 축소된 이세돌 왕관, 아니 황제관의 원작은 해루루와 연이랑 이라는 분임을 밝힙니다 TT: Lee Se-dol crown of the original, not the emperor shrink tube Oh Unveils haeru the base and opened yirang of bunim
한 바구니는 꿀꺽 ㅋ 아참 토마토 회장님이 겉과 속이 같아서 좋다고 하셧는데 ㅋ 토마토 종류가 엄청 많아요 A basket is to gulp Oh blah tomato hasyeotneun President and the outside like a good trick blah an awful lot kinds of tomatoes
ㅜ:: 아참 중국 게시판에 가보았는데요, 이창호에게 그렇게 지고도, 이창호에 대한 분노라든가, 이창호에 대한 공격같은 것은 찾아 볼 수가 없습니다 TT: Oh atneundeyo go to China Forum, it is also to Yichang, Yichang radeunga anger against, it is like attacking on the Yichang can not be found
우째 그렇게 삼성화재배 결과를 미리 잘 아는지 ㅋ 사실 대국 결과 한국 사이트보다 중국 사이트가 더 빠르고 정확합니다 ㅋㅋㅋ 아참 왕시가 중국 하남성 출신이라죠 In fact, I know in advance that you woojjae blah superpower China site faster and more accurate results than the well sit